In [21]:
import pandas as pd
import json

In [22]:
df_1 = pd.read_csv('../data/kaggle_resume_dataset.csv')

In [33]:
df_2 = pd.read_csv('../data/website_resumes.csv')

In [27]:
df_3 = pd.read_csv('../data/2480resumes.csv')

In [51]:
df_2['resume'][0].split('\n')

['\r',
 'GLENN WHITE\r',
 'Phone - 415-***-****\r',
 'Email - adn9fn@r.postjobfree.com\r',
 '\r',
 'PROFILE SUMMARY\r',
 '\r',
 '•8+ Years Android Application Development.\r',
 '•5 apps published on Google Play Store.\r',
 '•2 apps developed in Alpha phase.\r',
 '•1 app worked on for internal use.\r',
 '•Self-motivated and proactive Android Mobile Application Developer experienced working with different multi-disciplinary test-driven development (TDD) Agile teams on small to large mobile app projects.\r',
 '•Demonstrated ability to work and communicate effectively with other mobile engineers, device firmware engineers, technical leads, and project managers on teams applying Agile/Scrum methodologies.\r',
 '•Solid understanding of project and software development lifecycles and all the steps involved in moving an app development through to deployment on the Google App Store.\r',
 '•Applied various architecture and design patterns, including Singleton, MVP, REST, MVVP (Model, View, ViewM

In [62]:
train_data = pd.read_json('../test/traindata.json', lines=True)

In [63]:
train_data.shape

(200, 2)

In [26]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None


In [2]:
import spacy
import random

In [3]:
spacy.__version__

'3.2.2'

In [61]:
TRAIN_DATA[0]


('Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming

In [51]:
TRAIN_DATA = convert_dataturks_to_spacy('../test/testdata.json')
nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)


# add labels
for _, annotations in TRAIN_DATA:
     for ent in annotations.get('entities'):
        ner.add_label(ent[2])==0

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)

Statring iteration 0
{'ner': 1424.2040737901666}
Statring iteration 1
{'ner': 687.7199811311953}
Statring iteration 2
{'ner': 486.5370940342219}
Statring iteration 3
{'ner': 420.368192069885}
Statring iteration 4
{'ner': 359.82867820645635}
Statring iteration 5
{'ner': 316.63764802408326}
Statring iteration 6
{'ner': 270.9706268676556}
Statring iteration 7
{'ner': 220.03902189374904}
Statring iteration 8
{'ner': 194.1730694216207}
Statring iteration 9
{'ner': 171.37609418082937}


In [59]:
string = df_2.iloc[30]['resume']

In [60]:
nlp1 = spacy.load(r"../output/model-last") #load the best model
doc = nlp1(string)
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [42]:
nlp1 = spacy.load(r"../output/model-best") #load the best model
doc = nlp1("Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming Languages: C, C++, Java, .net, php.\n• Web Designing: HTML, XML\n• Operating Systems: Windows […] Windows Server 2003, Linux.\n• Database: MS Access, MS SQL Server 2008, Oracle 10g, MySql.\n\nhttps://www.indeed.com/r/Afreen-Jamadar/8baf379b705e37c6?isid=rex-download&ikw=download-top&co=IN") # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter